### Imports

In [ ]:
from decimal import Decimal
from collections import defaultdict
from dataclasses import dataclass
from typing import Dict, List, Generic, TypeVar, Optional
import math
from copy import deepcopy
from web3 import Web3
import json
import requests

### Retrieving Snapshot of Uniswap V2 from TheGraph API 

#### Query to retrieve the most recently-added liquidity pools

In [ ]:
import requests

query = """
{
  pairs(first: 5, orderBy: createdAtTimestamp, orderDirection: desc) {
    id
    token0 {
      symbol
    }
    token1 {
      symbol
    }
    createdAtTimestamp
  }
}
"""

thegraph_api_key = "87f5f40272b70e9df2648af0cf52b619"

headers = {
    "Authorization": f"Bearer {thegraph_api_key}",
    "Content-Type": "application/json"
}

# UNISWAP V2 Mainnet
thegraph_url = "https://gateway.thegraph.com/api/subgraphs/id/A3Np3RQbaBA6oKJgiwDJeo5T3zrYfGHPWFYayMwtNDum"

# Then compare w/ UNISWAP V2 Arbitrium...

response = requests.post(thegraph_url, json={'query': query}, headers=headers)

data = response.json()


In [ ]:
# Get reserve info on some pair

some_pair_id = data["data"]["pairs"][0]["id"]

query = f"""
{{
  pair(id: "{some_pair_id}") {{
    token0 {{ symbol name decimals id }}
    token1 {{ symbol name decimals id }}
    reserve0
    reserve1
    token0Price
    token1Price
    reserveUSD
    volumeUSD
    txCount
  }}
}}
"""

response = requests.post(url, json={'query': query}, headers=headers)

pair_data = response.json()

#### Query to retrieve all pairs currently on Uniswap V2

In [ ]:
# 2.1 Query to get total number of pairs

factory_query = """
{
  uniswapFactories(
    first: 1,
    where: { id: "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f" }
  ) {
    id
    pairCount
    totalVolumeUSD
  }
}
"""

resp = requests.post(url=thegraph_url, json={"query": factory_query}, headers=headers)
pair_count = int(resp.json()["data"]["uniswapFactories"][0]["pairCount"])
pair_count

In [ ]:
# 2.2 Queries to retrieve pair information

PAGE_SIZE = 1000
all_pairs = []

fields = """
id
createdAtTimestamp
createdAtBlockNumber
token0 { id symbol decimals }
token1 { id symbol decimals }
reserve0
reserve1
token0Price
token1Price
reserveUSD
trackedReserveETH
untrackedVolumeUSD
volumeUSD
totalSupply
liquidityProviderCount
txCount
"""

for skip in range(0, pair_count, PAGE_SIZE):
    page_query = f"""
    {{
        pairs(
            where: {{ trackedReserveETH_gt: "0" }}
            first: {PAGE_SIZE},
            skip: {skip},
            orderBy: trackedReserveETH,
            orderDirection: desc
        ) {{
        {fields}
        }}
    }}
    """
    r = requests.post(thegraph_url, json={"query": page_query}, headers=headers)
    data = r.json()["data"]["pairs"]
    all_pairs.extend(data)
    print(f"Fetched {len(data)} pairs (skip={skip})")

all_pairs

### Save & Reload Snapshot for offline testing

In [ ]:
output_path = "/Volumes/Extreme SSD/arbot_data/UniswapV2.json"

In [ ]:
with open(output_path, "w") as f:
    json.dump(all_pairs, f, indent=2)

In [ ]:
import json
with open(output_path, "r") as f:
    loaded_pairs = json.load(f)

#### Filtering/Preprocessing the liquidity pairs

In [ ]:
# Filtering conditions for pairs will go here...

trackedReserveETH_min = 10.0 # Set minimum reserve value size

filtered_pairs = []
for p in loaded_pairs:

    # 1) Zero reserves out
    if Decimal(p["reserve0"]) == Decimal(0.00) or Decimal(p["reserve1"]) == Decimal(0.00):
        continue
    
    # 2) Negligible size reserves out
    if Decimal(p["trackedReserveETH"]) < Decimal(trackedReserveETH_min):
        continue


    # Other filtering conditions here...


    filtered_pairs.append(p)

In [ ]:
len(filtered_pairs)

#### Constructing Graph; Defining Graph class

In [ ]:
# Declare global Uniswap V2 fees here
fee = Decimal(0.003)
F: Decimal = 1 - fee

# These may need to be scoped inside graph later
forward_weight  = lambda pair : Decimal(F) * (Decimal(pair["reserve1"]) / Decimal(pair["reserve0"]))
backward_weight = lambda pair : Decimal(F) * (Decimal(pair["reserve0"]) / Decimal(pair["reserve1"]))

@dataclass(frozen=True)
class Edge:
    source: str
    target: str
    weight: Decimal
    pool_id: str

class Graph:
    def __init__(self):
        self.nodes: Dict[str, dict] = {}
        self.adj: Dict[str, List[Edge]] = defaultdict(list)

    def add_pair(self, pair: dict):
        s, d = pair['token0'], pair['token1']
        id_s, id_d = s['id'], d['id']

        # store node data once
        self.nodes.setdefault(id_s, s)
        self.nodes.setdefault(id_d, d)

        # attach two directed edges
        self.adj[id_s].append(Edge(
            source=id_s,
            target=id_d,
            weight=forward_weight(pair),
            pool_id=pair['id'])
        )
        self.adj[id_d].append(Edge(
            source=id_d,
            target=id_s,
            weight=backward_weight(pair),
            pool_id=pair['id'])
        )
    
    def load_from_list(self, pairs: List[dict]):
        for p in pairs:
            self.add_pair(p)

g = Graph()
g.load_from_list(filtered_pairs)

#### Analyze the Graph

In [ ]:
# Sort by outdegree

outdeg = {node_id: len(edges) for node_id, edges in g.adj.items()}
sorted_by_outdeg = sorted(outdeg.items(), key=lambda kv: kv[1], reverse=True)

ranking = sorted_by_outdeg
symbols = [g.nodes[node_id]["symbol"] for node_id, _ in ranking ]
counts = [degree for _, degree in ranking]

In [ ]:
# Sort pairs by 'trackedReserveETH'
sorted_by_trackedReserveETH = sorted(filtered_pairs, key=lambda p: Decimal(p['trackedReserveETH']), reverse=True)

#### Prune the graph

In [ ]:
# Define few pruning/preprocessing functions
def min_outdeg(graph: Graph, min_outdeg) -> Graph:

    # Initialize some empty graph
    pruned = Graph()

    # Select "survining" nodes
    survivors = {node for node, edges in graph.adj.items() if len(edges) >= min_outdeg}

    # Copy node ids
    for node_id in survivors:
        pruned.nodes[node_id] = graph.nodes[node_id]
    
    # Copy edges iff both ends are in survivors...
    for src, edges in graph.adj.items():
        if src not in survivors:
            continue
        for e in edges:
            if e.target in survivors:
                pruned.adj[src].append(e)
    return pruned


# Other pruning funcs here...


pruned_g = min_outdeg(g, min_outdeg=2)


In [ ]:
len(pruned_g.nodes)

#### Preprocess the graph

In [ ]:
def to_nlog_graph(graph: Graph) -> Graph:

    # Start with new graph from scratch
    logg = Graph()

    # Copy all nodes
    logg.nodes = graph.nodes.copy()

    # Rebuild new edges with transformed weights
    for src, edges in graph.adj.items():
        for e in edges:
            w = float(e.weight)
            log_w = -math.log(w)
            logg.adj[src].append(Edge(
                source=e.source,
                target=e.target,
                weight=Decimal(log_w),
                pool_id=e.pool_id
            ))
    return logg



# Other preprocessing funcs here ...
# e.g. "bake in" gas fees for each transition etc.


nlog_g = to_nlog_graph(pruned_g)

#### Modified Bellman-Ford on Graph Implementation

In [ ]:
### Helpers ###

# Map log weights back to original weights
def map_to_pruned_edges(pruned_g: Graph,
                        log_cycle: List[Edge]
                       ) -> List[Edge]:
    real_cycle: List[Edge] = []
    for le in log_cycle:
        matches = [e for e in pruned_g.adj[le.source]
                   if e.target == le.target and e.pool_id == le.pool_id]
        real_cycle.append(matches[0])
    return real_cycle


def cycle_product(edges: List[Edge]) -> Decimal:
    # Simple naive, not handling exceptions
    total = Decimal(1)
    for e in edges:
        total *= e.weight
    return total

def cycle_symbols(graph: Graph, edges: List[Edge]) -> List[str]:
    return [graph.nodes[some_edge.source]["symbol"] for some_edge in edges] + [graph.nodes[edges[-1].target]["symbol"]]


### BF ###

# For now, only impose max_len cycle length as only modification
# Algo will be later further modified...

def find_negative_cycle(
    graph: Graph,
    max_len: int
) -> Optional[List[Edge]]:
    """
    Bellman–Ford with a supersource; returns a negative cycle as a list of Edge
    (in forward order), or None if no cycle of length <= max_len exists.
    """
    # 1) Initialize distances and predecessor‐edges
    dist = {node: Decimal(0) for node in graph.nodes}
    pred_edge: dict[str, Optional[Edge]] = {node: None for node in graph.nodes}

    # 2) Relax edges up to max_len times
    for _ in range(max_len):
        updated = False
        for u, edges in graph.adj.items():
            for e in edges:
                v = e.target
                if dist[u] + e.weight < dist[v]:
                    dist[v] = dist[u] + e.weight
                    pred_edge[v] = e
                    updated = True
        if not updated:
            break

    # 3) Check for any further relaxable edge ⇒ negative cycle
    for u, edges in graph.adj.items():
        for e in edges:
            v = e.target
            if dist[u] + e.weight < dist[v]:
                # We found a cycle; now reconstruct it as a list of Edges
                # Step A: walk back max_len hops to ensure we land inside the cycle
                cur = v
                for _ in range(max_len):
                    cur = pred_edge[cur].source

                # Step B: collect edges until we loop back to `cur`
                start = cur
                cycle_edges: List[Edge] = []
                while True:
                    edge_back = pred_edge[cur]
                    cycle_edges.append(edge_back)
                    cur = edge_back.source
                    if cur == start:
                        break

                # Step C: reverse to get the forward traversal order
                cycle_edges.reverse()
                return cycle_edges
    return None


# Recursive func removing one edge each time, to enumerate all the cycles
def enumerate_cycles(
        graph: Graph,
        max_len: int,
    ) -> List[List[Edge]]:

    g = deepcopy(graph)
    cycles: List[List[Edge]] = []

    while True:
        cycle = find_negative_cycle(g, max_len)
        if cycle is None:
            break
        cycles.append(cycle)

        # Remove first edge of cycle from graph and then re-run to find another fresh cycle
        e = cycle[0]
        g.adj[e.source] = [
            edge for edge in g.adj[e.source]
            if not (edge.target == e.target and edge.pool_id == e.pool_id)
        ]
    
    return cycles


# Find all cycles at practically unbounded length
log_cycles = enumerate_cycles(nlog_g, max_len=100)
real_cycles = [map_to_pruned_edges(pruned_g, edges) for edges in log_cycles]

# Print out the symbols for clarity
[cycle_symbols(pruned_g, edges) for edges in log_cycles]

In [ ]:
# Check all nodes connected to "USD" giving anomaly
[node for node in pruned_g.nodes if pruned_g.nodes[node]["symbol"] == "USD"]
[pruned_g.nodes[edge.target]["symbol"] for edge in pruned_g.adj['0xd233d1f6fd11640081abb8db125f722b5dc729dc']]

In [ ]:
# Check if cycle products add up
[cycle_product(edges) for edges in real_cycles]

#### Simulate Cycle Swap Execution on EVM UniswapV2 Router Contract / CFMM

In [ ]:
# Self-calculation: Probably has some rounding defects...

get_pair = lambda pool_id : next((pair for pair in filtered_pairs if pair["id"] == pool_id), None)

def get_reserve(some_edge, node_id) -> Decimal:
    pair = get_pair(some_edge.pool_id)

    if pair["token0"]["id"] == node_id:
        return Decimal(pair["reserve0"])
    else:
        return Decimal(pair["reserve1"])

swap = lambda x, y, dx : Decimal(F * y * dx) / Decimal(x + F * dx)

def my_swap(edges: List[Edge], dx):

    curr_input = dx

    for some_edge in edges:
        x = get_reserve(some_edge, some_edge.source)
        y = get_reserve(some_edge, some_edge.target)

        curr_out = swap(x, y, curr_input)
        curr_input = curr_out
    
    return curr_out


# Try fetch fraction of reserve
test_cycle = real_cycles[-3]
start_edge = test_cycle[0]

dx = int(get_reserve(start_edge, start_edge.source) / 1000)

dx, my_swap(test_cycle, dx)

In [ ]:
# EVM UniswapV2 Router Contract calculation: uses integer math for the multiplications

# 0. set up RPC w3 connection
api_key = "75db4a2f907d4525866a728681b3b458"
infura_url = f"https://mainnet.infura.io/v3/{api_key}" # select mainnet or sepolia here
w3 = Web3(Web3.HTTPProvider(infura_url))

# 1. Load the Router contract using etherscan API
def fetch_abi(address: str) -> list:
    ETHERSCAN_API_KEY = "QGS1VSEPNEHZ8W4M686QCNM1Z6WGIT14Q1"
    url = (
      "https://api.etherscan.io/api"
      f"?module=contract&action=getabi&address={address}"
      f"&apikey={ETHERSCAN_API_KEY}"
    )
    resp = requests.get(url).json()
    return json.loads(resp["result"])

router_address = Web3.to_checksum_address("0x7a250d5630b4cf539739df2c5dacb4c659f2488d")
router_abi     = fetch_abi(router_address)
router         = w3.eth.contract(address=router_address, abi=router_abi)

# 2. Router swap routine, taking already-instantiated outer router contract
def router_swap(edges: List[Edge], graph: Graph, dx: float):
    # Generate list of intermediary tokens
    tokens: List[dict] = [graph.nodes[some_edge.source] for some_edge in edges]
    tokens.append(graph.nodes[edges[-1].target]) # final token to wrap up path

    # Router requires a path of contract addresses, and an initial integer input amount
    path = [Web3.to_checksum_address(token["id"]) for token in tokens]
    amounts = router.functions.getAmountsOut( int(dx * 10 ** int(tokens[0]["decimals"])) , path).call()

    return amounts [-1] / 10 ** int(tokens[-1]["decimals"])
  

# 3. Call here with same dx value set above
dx, router_swap(test_cycle, pruned_g, dx)

### Geth client setup

In [ ]:
my_password = "secure_password"
keystore_path = "keystore.json"

with open(keystore_path, "r") as f:
    keystore_json = json.load(f)

ex_privkey = w3.eth.account.decrypt(keystore_json, my_password)
ex_account = w3.eth.account.from_key(ex_privkey)

my_account = ex_account